In [2]:
import csv
import rasterio
import os
import numpy as np
from pathlib import Path, PurePath

In [3]:
import os
import inspect
import dl_toolbox
inspect.getfile(dl_toolbox)

'/home/eh/fournip/dl_toolbox/dl_toolbox/__init__.py'

In [4]:
if os.uname().nodename == 'WDTIS890Z': 
    data_root = Path('/mnt/d/pfournie/Documents/data')
    home = Path('/home/pfournie')
    save_root = data_root / 'outputs'
elif os.uname().nodename == 'qdtis056z': 
    data_root = Path('/data')
    home = Path('/d/pfournie')
    save_root = data_root / 'outputs'
else:
    data_root = Path('/work/OT/ai4geo/DATA/DATASETS')
    #data_root = Path(os.environ['TMPDIR'])
    home = Path('/home/eh/fournip')
    save_root = Path('/work/OT/ai4usr/fournip') / 'outputs'

In [4]:
# Split SEMCITY   
data_path = data_root / 'SemCity-Toulouse-bench'

with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/semcity_16tiles.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(
        [
            'city',
            'tile_id',
            'img_path',
            'label_path',
            'x0',
            'y0',
            'patch_width',
            'patch_height',
            'fold_id',
            'min_vals',
            'max_vals'
        ]
    )
    for i in range(16): 
        image_path = 'img_multispec_05/TLS_BDSD_M/TLS_BDSD_M_' + f'{i+1:02}' + '.tif'
        label_path = 'semantic_05/TLS_GT/TLS_GT_' + f'{i+1:02}' + '_1.tif'
        with rasterio.open(data_path/image_path) as f:
            height, width = f.shape
            stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
            # for tile in get_tiles(width, height, size=876, size2=863):
            #     writer.writerow([
            #         'SemcityToulouse',
            #         i,
            #         image_path,
            #         label_path,
            #         tile.col_off,
            #         tile.row_off,
            #         tile.width,
            #         tile.height,
            #         i % 64
            #     ])
            writer.writerow(
                [
                    'Semcity',
                    i,
                    image_path,
                    label_path,
                    0,
                    0,
                    width,
                    height,
                    i,
                    str(tuple([round(s.min,3) for s in stats])),
                    str(tuple([round(s.max,3) for s in stats]))
                ]
            )

In [16]:
with rasterio.open(digitanie/'Toulouse'/f'Toulouse_EPSG32631_4.tif') as f:
    print('small tile bounds : ', f.bounds)
    x = 373000
    y = 4828000
    print(f'point of coords {(x,y)} at indices : ', f.index(x,y))
    print('point value : ', next(f.sample([(x,y)])))
    
with rasterio.open(digitanie/'Toulouse'/f'Toulouse.tif') as f:
    print(f'point of coords {(x,y)} at indices {f.index(x,y)} in big tile has value', next(f.sample([(x,y)])))


small tile bounds :  BoundingBox(left=372116.3286738682, bottom=4827134.503163704, right=373140.30450558936, top=4828158.478995425)
point of coords (373000, 4828000) at indices :  (316, 1767)
point value :  [0.04579515 0.06582139 0.0960011  0.10355585]
point of coords (373000, 4828000) at indices (30436, 29346) in big tile has value [0.07543897 0.09317079 0.10472748 0.28273705]


In [5]:
src = rasterio.open(Path('/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Toulouse/COS9/Toulouse_4.tif'))
img = src.read(out_dtype=np.uint8)
print(np.unique(img.reshape(img.shape[0], -1), axis=1, return_counts=True))

(array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=uint8), array([ 982908,   19568,  397671,     477, 1180129,  655259,  213706,
        599269,  136140,    9177]))


In [5]:
# Split DIGITANIE ALL

digitanie = data_root / 'DIGITANIE'

cities = [
    'Arcachon',
    'Brisbane',
    'Can-Tho',
    'Lagos',
    'Maros',
    'Munich',
    'Paris',
    'Rio-Janeiro',
    'Strasbourg',
    'Toulouse',
    'Biarritz',
    'Buenos-Aires',
    'Helsinki',
    'Le-Caire',
    'Montpellier',
    'Nantes',
    'Port-Elisabeth',
    'Shanghai',
    'Tianjin'
]

i = 0
with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/digitanie_all.csv', 'w+', newline='') as csvfile:

    writer = csv.writer(csvfile)
    writer.writerow(
        [
            'city',
            'tile_id',
            'img_path',
            'label_path',
            'x0',
            'y0',
            'patch_width',
            'patch_height',
            'fold_id',
            'min_vals',
            'max_vals'
        ]
    )
    
    for city in cities:
        print(city)
        # Opening and computing stats and writing from big city image
        big_tile_path = digitanie/city/f'{city}.tif'        
        with rasterio.open(big_tile_path) as f:
            height, width = f.shape
            big_tile = f.read(window=rasterio.windows.Window(20000, 20000, 10000, 10000))[..., ::5, ::5]
            p2, p98 = np.percentile(big_tile, (2, 98), axis=(1,2))
            print(p2)
            print(p98)
            #stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
        writer.writerow(
            [
                f'Digitanie',
                0,
                big_tile_path.relative_to(digitanie),
                'none',
                0,
                0,
                width,
                height,
                i,
                str(tuple(p2)),
                str(tuple(p98))
            ]
        ) 
        i+=1

        # List of small tiles
        img_paths = sorted(
            [path.relative_to(digitanie) for path in Path(digitanie/city).glob('*_[0-9].tif')], 
            key=lambda x: int(x.stem.split('_')[-1])
        )
        msk_paths = sorted(
            [path.relative_to(digitanie) for path in Path(digitanie/city/'COS9').glob('*.tif')],
            key=lambda x: int(x.stem.split('_')[1][0])
        )
        
        # Creating the csv
        for img, msk in zip(img_paths, msk_paths):
            with rasterio.open(digitanie/img) as f:
                height, width = f.shape
                tile = f.read()
                p2, p98 = np.percentile(tile, (2, 98), axis=(1,2))
                print(p2)
                print(p98)
                #stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
                writer.writerow(
                    [
                        f'Digitanie',
                        0,
                        img,
                        msk,
                        0,
                        0,
                        width,
                        height,
                        i,
                        str(tuple(p2)),
                        str(tuple(p98))
                    ]
                )
                i+=1


Arcachon
[0.02960374 0.05438247 0.07169499 0.01372184]
[0.31214613 0.2590517  0.21798451 0.36978838]
[0.03412274 0.05282458 0.06949054 0.02345102]
[0.33827722 0.27952224 0.23244344 0.37972685]
[0.03232118 0.05033357 0.06746248 0.0900485 ]
[0.28122536 0.23452839 0.19942535 0.38389132]
[0.03064246 0.05091313 0.06891563 0.01590963]
[0.24313749 0.19614465 0.18387327 0.36523966]
[0.03016255 0.04827472 0.06579828 0.09412559]
[0.13173872 0.12314086 0.12523928 0.3292416 ]
[0.03169039 0.04875289 0.06658392 0.01841799]
[0.29819008 0.23972902 0.21242174 0.41209144]
[0.03212715 0.04967954 0.06797625 0.06433858]
[0.25940626 0.20858724 0.19326142 0.39648689]
[0.02901955 0.04741305 0.06519303 0.09405353]
[0.13743877 0.10750752 0.11186822 0.35966659]
[0.03276377 0.04908641 0.06741016 0.04296031]
[0.31809145 0.26017412 0.22289539 0.41951294]
[0.03704559 0.05466623 0.0714739  0.07644182]
[0.28652625 0.22783074 0.21040617 0.41645946]
[0.0334366  0.05269316 0.06969865 0.03716102]
[0.27334858 0.21734849 0.

In [7]:
if os.uname().nodename == 'WDTIS890Z':
    data_path = Path('/mnt/d/pfournie/Documents/data')
elif os.uname().nodename == 'qdtis056z':
    data_path = Path('/data')
else:
    data_path = Path('/work/OT/ai4geo/DATA/DATASETS')
    
digitanie = data_path / 'DIGITANIE'
cities = ['Toulouse']
with open(Path.home() / f'dl_toolbox/dl_toolbox/datamodules/digitanie_toulouse_big_2.csv', 'w+', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(
        [
            'city',
            'tile_id',
            'img_path',
            'label_path',
            'x0',
            'y0',
            'patch_width',
            'patch_height',
            'fold_id',
            'min_vals',
            'max_vals'
        ]
    )
    for i, city in enumerate(cities):
        big_tile_path = digitanie/city/f'{city}.tif'        
        with rasterio.open(big_tile_path) as f:
            height, width = f.shape
            stats = [f.statistics(bidx=i, approx=True, clear_cache=False) for i in range(1, f.count+1)]
            writer.writerow(
                [
                    f'Digitanie',
                    i,
                    big_tile_path.relative_to(digitanie),
                    'none',
                    0,
                    0,
                    width,
                    height,
                    i,
                    str(tuple([round(s.min,3) for s in stats])),
                    str(tuple([round(s.max,3) for s in stats]))
                ]
            )

Warning 1: Unable to save auxiliary information in /work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Toulouse/Toulouse.tif.aux.xml.
